In [ ]:
import cv2
import pytesseract
import numpy as np

Tesseract 경로 설정 (시스템에 맞게 경로 수정)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 이미지 읽기
image = cv2.imread('your_image_path.jpg')

# 1. 그레이스케일 변환
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 2. 가우시안 블러 적용
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# 3. 이진화 (Adaptive Thresholding을 사용하여 텍스트 강조)
binary_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# 4. 대비 향상 (CLAHE 사용)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(binary_image)

# 5. 엣지 검출 (Canny Edge Detection)
edges = cv2.Canny(enhanced_image, 100, 200)

# 6. 텍스트 영역 검출을 위한 외곽선 찾기
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 텍스트 검출한 영역을 표시 (Bounding Box)
for contour in contours:
    # 외곽선으로부터 경계 상자를 계산
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    
    # 텍스트로 추정되는 영역 필터링 (너무 좁거나 넓은 영역 제외)
    if 0.5 < aspect_ratio < 5 and w > 30 and h > 15:
        # 검출한 텍스트 영역에 사각형 그리기
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # 7. 검출한 영역을 잘라 OCR 수행
        roi = enhanced_image[y:y + h, x:x + w]
        text = pytesseract.image_to_string(roi, lang='eng')
        print(f"Detected text: {text.strip()}")
        
# 결과 이미지 출력
cv2.imshow("Text Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
